# ChatDEV

This Notebook will describe the implementation of ChatDEV, a simulated software company based on AI agents.

### How to use it?

- (If needed) Select a Kernel, typically Python 3.10.2
- (If needed) Use "Clear All Outputs"
- Use "Run All" to run all code blocks after another
- Scroll to the very bottom to see the output
  - You might need to click on "View as a scrollable element" at the bottom to see all output

## 1. Initialize environment

We use the `dotenv` package and its `load_dotenv` method to load `.env` configuration files.

We always load the (always existing) `../.env` file and override it with a local
`../.env.local` file.

This avoids leaking secrets to the repository, e.g. the GPT-4 API Key.

In [20]:
from dotenv import load_dotenv

load_dotenv("../.env")
load_dotenv("../.env.local", override=True)

True

## 2. Initialize OpenAI Client

We create an OpenAI API client and load two values from our env files: the LLM model of Open AI to use (normally gpt-4-turbo) and an assistant ID, which specifies which "GPT", a specific personality of ChatGPT, we want to use.

The "GPT" (Assistant) has been created in the OpenAI GPT management panel beforehand.

In [21]:
from openai import OpenAI
from os import getenv

# Automatically loads OPENAI_API_KEY
oai_client = OpenAI()
oai_model = getenv('OPENAI_API_MODEL')
oai_assistant_id = getenv('OPENAI_API_ASSISTANT_ID')

## 3. Domain Objects

In order to properly represent our workflow, we need a bunch of domain objects:

- **WorkflowRole** - A specific agent/role that is part of a workflow
- **WorkflowArtifact** - An input or output created by a conversation of a workflow
- **WorkflowConversation** - A conversation between two roles. They take an input and and output artifact.
- **WorkflowPhase** - A phase of a workflow. It is a logical group of conversations.

These domain objects act as the basis for configuring workflows and letting them be handled by AI agents.

In [22]:
from typing import List, Set, Optional

class WorkflowRole:
  """
  A workflow role describes a specific agent or person inside a workflow.
  
  Examples
  --------

  WorkflowRole("CEO", "CEO of the company")
  WorkflowRole("Developer", "Developer working on the code")
  """

  def __init__(self, name: str, description: str):
    """ 
    Creates a new workflow role with the given name and description.
    """
    self.name = name
    self.description = description
  
  def gpt_str(self) -> str:
    """
    This is the string representation that ChatGPT will receive
    """
    return f"('role' name:[{self.name}], description:[{self.description}])"

  def __str__(self) -> str:
    """
    This is the string representation for debugging or errors
    """
    return f"WorkflowRole[{self.name}]"

class WorkflowArtifact:
  """
  A workflow artifact describes inputs and outputs of conversations.
  
  Examples
  --------
  WorkflowArtifact("Requirements", "Requirements for the code")
  WorkflowArtifact("Code", "The code that was written")
  """

  def __init__(self, name: str, description: str):
    """
    Creates a new workflow artifact with the given name and description.
    """
    self.name = name
    self.description = description
  
  def gpt_str(self) -> str:
    """
    This is the string representation that ChatGPT will receive
    """
    return f"('artifact' name:[{self.name}], description:[{self.description}])"

  def __str__(self) -> str:
    """
    This is the string representation for debugging or errors
    """
    return f"WorkflowArtifact[{self.name}]"

class WorkflowConversation:
  """
  A workflow conversation describes a conversation between two roles in a workflow.

  Examples
  --------
  WorkflowConversation(
    "Code Review",
    "Code review between developer and CEO",
    WorkflowRole("Developer", "Developer working on the code"),
    WorkflowRole("CEO", "CEO of the company"),
    WorkflowArtifact("Code", "The code that was written"),
    WorkflowArtifact("Requirements", "Requirements for the code")
  )
  """
  def __init__(self, name: str, description: str, lead: WorkflowRole, assistant: WorkflowRole, input: WorkflowArtifact, output: WorkflowArtifact):
    """
    Creates a new workflow conversation with the given name, description, roles, and artifacts.
    """
    self.name = name
    self.description = description
    self.lead = lead
    self.assistant = assistant
    self.input = input
    self.output = output

  def roles(self) -> Set[WorkflowRole]:
    """
    Returns a set of all roles involved in this conversation.
    """
    return {self.lead, self.assistant}
  
  def gpt_str(self) -> str:
    """
    This is the string representation that ChatGPT will receive
    """
    return f"('conversation' name:[{self.name}], description:[{self.description}], lead name:[{self.lead.name}], assistant name:[{self.assistant.name}], input name:[{self.input.name}], output name:[{self.output.name}])"

  def __str__(self) -> str:
    """
    This is the string representation for debugging or errors
    """
    return f"WorkflowConversation[{self.name}]"

class WorkflowPhase:
  """
  A workflow phase describes a phase in a workflow, which consists of multiple conversations.

  Examples
  --------
  WorkflowPhase(
    "Code Review",
    "Code review between developer and CEO",
    [
      WorkflowConversation(
        "Code Review",
        "Code review between developer and CEO",
        WorkflowRole("Developer", "Developer working on the code"),
        WorkflowRole("CEO", "CEO of the company"),
        WorkflowArtifact("Code", "The code that was written"),
        WorkflowArtifact("Requirements", "Requirements for the code")
      ),
      WorkflowConversation(
        "Code Review",
        "Code review between developer and CEO",
        WorkflowRole("Developer", "Developer working on the code"),
        WorkflowRole("CEO", "CEO of the company"),
        WorkflowArtifact("Code", "The code that was written"),
        WorkflowArtifact("Requirements", "Requirements for the code")
      )
    ]
  )
  """
  def __init__(self, name: str, description: str, conversations: List[WorkflowConversation]):
    """
    Creates a new workflow phase with the given name, description, and conversations.
    """
    self.name = name
    self.description = description
    self.conversations = conversations
    self.current_conversation_index = 0
  
  def roles(self) -> Set[WorkflowRole]:
    """
    Returns a set of all roles involved in this phase.
    """
    return {role for conversation in self.conversations for role in conversation.roles()}
  
  def current_conversation(self) -> WorkflowConversation:
    """
    Returns the current conversation in this phase.
    """
    if self.ended():
      raise RuntimeError(f"Conversation {self.name} already ended")
    return self.conversations[self.current_conversation_index]
  
  def next_conversation(self):
    """
    Moves to the next conversation in this phase.
    """
    if self.ended():
      raise RuntimeError(f"Conversation {self.name} already ended")

    self.current_conversation_index += 1
  
  def ended(self):
    """
    Returns whether this phase has ended.
    """
    return self.current_conversation_index >= len(self.conversations)
  
  def gpt_str(self) -> str:
    """
    This is the string representation that ChatGPT will receive
    """
    return f"('workflow' name:[{self.name}], description:[{self.description}], conversation names:[{','.join([conversation.name for conversation in self.conversations])}])"

  def __str__(self) -> str:
    """
    This is the string representation for debugging or errors
    """
    return f"WorkflowPhase[{self.name}]"


## 4. AI Threads

We create an implementation called `AiThread` which represents a single, long thread or conversation an AI has.

ChatGPT provides an own API for that which allows us to create threads, add messages and then let the AI answer
to our thread.

It's API is basically pretty simple:

```python
thread = AiThread(...)

response = thread.send("Hey ChatGPT, say 'CHEESE'!")
print(response) # "CHEESE"

next_response = thread.send("What did I tell you to say?")
print(response) # "You told me to say CHEESE"

thread.destroy()
```

In [23]:
from openai.types.beta.thread import Thread
from time import sleep

class AiThread:
  """
  An AiThread represents a thread in the OpenAI API.

  We can send messages to it and receive the response to that message.
  """

  def __init__(self, oai_client: OpenAI, model: str, assistant_id: str, instructions: str, initial_message: str):
    """
    Creates a new AiThread with the given OpenAI client, model, assistant ID, instructions, and initial message.
    """
    self.oai_client = oai_client
    self.model = model
    self.assistant_id = assistant_id
    self.instructions = instructions
    self.initial_message = initial_message
    self.current_thread: Optional[Thread] = None
    self.last_message_id: Optional[str] = None

  def thread(self) -> Thread:
    """
    Returns the current OpenAI thread.
    """
    if self.current_thread is None:
      self.current_thread = self.oai_client.beta.threads.create(
        messages= [{
          "role": "user",
          "content": self.initial_message
        }]
      )

    return self.current_thread
  
  def send(self, content: str) -> str:
    """
    Sends a message to the thread and returns the response.
    """
    message = self.oai_client.beta.threads.messages.create(
      thread_id= self.thread().id,
      role= "user",
      content= content,
    )

    run = self.oai_client.beta.threads.runs.create(
      thread_id= self.thread().id,
      model= self.model,
      assistant_id= self.assistant_id,
      instructions= self.instructions,
    )

    while run.status == 'queued' or run.status == 'in_progress' or run.status == 'requires_action':
      run = self.oai_client.beta.threads.runs.retrieve(run_id= run.id, thread_id= self.thread().id)
      if run.status != 'queued' and run.status != 'in_progress' and run.status != 'requires_action':
        break

      sleep(5)

    new_messages = self.oai_client.beta.threads.messages.list(
      thread_id= self.thread().id,
      limit= 1,
      before= message.id,
    )

    if len(new_messages.data) == 0:
      raise RuntimeError(f"No new messages found in thread {self.thread().id}")
    
    return new_messages.data[0].content[0].text.value
  
  def destroy(self):
    """
    Destroys the thread.
    """
    if self.current_thread is None:
      return
    
    self.oai_client.beta.threads.delete(thread_id= self.current_thread.id)


## 5. Workflow Manager

The workflow manager is the engine. It keeps track of the current workflow phase, the current conversation, and the current roles.

It also keeps track of the current thread and sends messages to it.

It will send very specific messages or react to specific keywords in messages.

Before each message it sends it will do a sanity check. It will instruct the agents to switch by using the "SWITCH" keyword.
It does so by sending a large set of instructions for everyone with it.

Execution of a workflow is pretty easy, in essence:

```python
manager = WorkflowManager(...)

manager.execute(
  """
  I'll have two number 9s, a number 9 large, a number 6 with extra dip, a number 7, two number 45s, one with cheese, and a large soda. 
  """
)
```

The manager will then go and print the whole conversation step by step as it unfolds.

In [24]:
class WorkflowManager:
  """
  A WorkflowManager manages a workflow.

  It keeps track of the current phase and conversation and provides methods to move to the next one.
  """
  def __init__(self, oai_client: OpenAI, model: str, assistant_id: str, description: str, phases: List[WorkflowPhase]):
    """
    Creates a new WorkflowManager with the given OpenAI client, model, assistant ID, description, and phases.
    """
    self.oai_client = oai_client
    self.model = model
    self.assistant_id = assistant_id
    self.description = description
    self.phases = phases
    self.current_phase_index = 0
    self.last_message_id: Optional[str] = None

  def roles(self) -> Set[WorkflowRole]:
    """
    Returns a set of all roles involved in this workflow.
    """
    return {role for phase in self.phases for role in phase.roles()}

  def conversations(self) -> Set[WorkflowConversation]:
    """
    Returns a set of all conversations involved in this workflow.
    """
    return {conversation for phase in self.phases for conversation in phase.conversations}
  
  def current_phase(self) -> WorkflowPhase:
    """
    Returns the current phase in this workflow.
    """
    if self.ended():
      raise RuntimeError(f"Workflow already ended")
    return self.phases[self.current_phase_index]
  
  def next_phase(self):
    """
    Moves to the next phase in this workflow.
    """
    if self.ended():
      raise RuntimeError(f"Workflow ended")
    self.current_phase_index += 1
  
  def ended(self) -> bool:
    """
    Returns whether this workflow has ended.
    """
    return self.current_phase_index >= len(self.phases)
  
  def current_conversation(self) -> WorkflowConversation:
    """
    Returns the current conversation in this workflow.
    """
    return self.current_phase().current_conversation()

  def next_conversation(self):
    """
    Moves to the next conversation in this workflow.
    """
    self.current_phase().next_conversation()

  def phase_ended(self) -> bool:
    """
    Returns whether the current phase has ended.
    """
    return self.current_phase().ended()
  
  def execute(self, input: str):
    """
    Executes the workflow with the given input.
    """

    eol = '\n'
    instructions = '''
      You are roleplaying a multiple people in a workflow, e.g. a company working on things in a process.
      In each response you always only represent a single person.
      The roleplay is structured in phases and each phase has a conversation.
      Each conversation always has a lead and an assistant role you are to impersonate.
      Each conversation has an input type and an output type you will try to create.
      Use all existing inputs and outputs created to properly create the respective output.

      In my initial message I will provide instructions that are to be handled by the workflow based on the input of the conversation.
      I provide you with descriptions of the conversations and the phases they belong to so that you know what to do.
  
      When I say "START PHASE <Phase Name>" you will realize you are now in the named phase of the workflow.
      You will respect what the phase is about, what the participating roles are and what the result should be.
      If you understood everything in that phase say only "SUCCESS", if not say only "FAILURE"

      When I say "START CONVERSATION <Conversation Name>" you will impersonate the lead of the conversation.
      You will read the existing thread and everything it it carefully.
      You will find an input per description. You answer to the provided assistant in respect to the roles and inputs described.

      When I say "SWITCH" you will impersonate the assistant, read the whole thread and answer to the lead.

      When I say "SWITCH" again you will impersonate the lead, read the whole thread again and answer to the assistant.

      Each impersonation can explain things or ask questions.
      If you feel like you are done, your message will only contain the conversation's desired
      output and end with "END CONVERSATION".

      Always start your responses with the name of the role, e.g.:

      ```
      CEO:

      <the message>
      ```

      ```
      CPO:

      <the message>
      ```
    '''

    initial_message = f'''
      These are the roles you are to impersonate:
      {eol.join([role.gpt_str() for role in self.roles()])}

      These are the artifacts that are referenced as inputs and outputs in conversations:
      {eol.join([role.gpt_str() for role in self.roles()])}

      These are the conversations you are to hold. They are referenced in phases:
      {eol.join([conversation.gpt_str() for conversation in self.conversations()])}

      These are the phases you are going through:
      {eol.join([phase.gpt_str() for phase in self.phases])}

      Here is a general description of the workflow for context:
      {self.description}

      This is the instruction you will handle with the workflow:
      ({input})
    '''

    thread = AiThread(
      oai_client= self.oai_client,
      model= self.model,
      assistant_id= self.assistant_id,
      instructions= instructions,
      initial_message= initial_message,
    )

    # Sanity check
    response = thread.send('''
      Answer with only "SUCCESS" if you understood everything and with only "FAILURE" if you didn't."
    ''')

    if response != "SUCCESS":
      raise RuntimeError(f"Sanity check failed. Response: {response}")

    while not self.ended():
      phase = self.current_phase()

      print(f"\n=== Phase {phase.name} ===\n\n")
      last_response = thread.send(f'''
        START PHASE {phase.name}
      ''')
      if last_response != "SUCCESS":
        raise RuntimeError(f"Failed start of phase {phase.name}")

      while not self.phase_ended():
        conversation = self.current_conversation()
        print(f"\n=== Conversation {conversation.name} ===\n\n")
        last_response = thread.send(f'''
          START CONVERSATION {self.current_conversation().name}
        ''')
        print(f"{last_response}\n")
        print("=====\n")
      
        message_count = 0
        while message_count < 10 and "END CONVERSATION" not in last_response:
          last_response = thread.send("SWITCH")
          print(f"{last_response}\n")
          print("=====\n")
          message_count += 1

        self.next_conversation()
      self.next_phase()

    thread.destroy()

  def __str__(self) -> str:
    return f"WorkflowManager[{self.name}]"

## 6. Workflow configuration

Now we have all required pieces to put together our full workflow.

For that we use our domain objects and define and link all important information we can think of.

### 6.1. Roles

We have 7 roles in total that occur in different conversations, sometimes multiple times, sometimes only once:

- **CEO**
- **CPO**
- **CTO**
- **Programmer**
- **Designer**
- **Reviewer**
- **Tester**

In [25]:
ceo = WorkflowRole(
  name= 'CEO',
  description= '''
    You are the CEO of a business company. You understand business values, resources, management
    and the business side of software development.
  ''',
)

cpo = WorkflowRole(
  name= 'CPO',
  description= '''
    You are the CPO of the company. You understand the product and the business side of software
    development. You represent the bridge between the business roles CEO and Customer and the
    technical roles CTO and Programmer.
  ''',
)

cto = WorkflowRole(
  name= 'CTO',
  description= '''
    You are the CTO of the company.
    You understand the project from a technical perspective and represent the bridge between
    the technical team and the business roles CEO, CPO and Customer.

    You are an expert in software development and understand all best practices.
    You can program really well and understand software architecture.
  ''',
)

programmer = WorkflowRole(
  name= 'Programmer',
  description= '''
    You are an intermediate programmer.
    You can program really well, but not perfect. You don't understand all best practices yet,
    but the code you produce works and is solid.

    You understand software architecture and can implement it.
  ''',
)

designer = WorkflowRole(
  name= 'Designer',
  description= '''
    You are a software designer.
    You understand software architecture well, understand UX and a lot of business properties
    of software as well as common best practices.

    You can design software architecture and can implement it.
  ''',
)

reviewer = WorkflowRole(
  name= 'Reviewer',
  description= '''
    You are a code reviewer.
    You understand code from the inside out and can spot problems with an implementation.
    You write review tasks that the programmer can then use to improve the code.
  ''',
)

tester = WorkflowRole(
  name= 'Tester',
  description= '''
    You are a software tester.
    You test code, uncover and problems that the programmer can resolve.
    You write test concepts that the programmer can implement.
  ''',
)

### 6.2. Artifacts

We have a lot of different artifacts that occur between conversations. Each artifact will evolve the next one:

**Task** > **Modalities** > **Language** > **Code** > **Designed Code** > **Reviewed Code** > **Tested Code** > **Spec** > **Manual**

In [26]:
task = WorkflowArtifact(
  name= 'Task',
  description= '''
    The business task as defined by the customer. It should be realised as a software product or feature.
  ''',
)

modalities = WorkflowArtifact(
  name= 'Modalities',
  description= '''
    The modalities of the task as defined by CEO and CPO. It defines the time for development, the budget,
    human resource costs and similar constraints.
  ''',
)

language = WorkflowArtifact(
  name= 'Language',
  description= '''
    The use cases of the task defined in human language which can be translated between
    business and technical department easily.
  '''
)

code = WorkflowArtifact(
  name= 'Code',
  description= '''
    The code as written by the programmer. It is yet to be designed, reviewed and tested.
  ''',
)

designed_code = WorkflowArtifact(
  name= 'Designed Code',
  description= '''
    The code as enhanced by the designer. It takes into account architecturial decisions
    and things like UX, best practices etc. It is yet to be reviewed and tested.
  ''',
)

reviewed_code = WorkflowArtifact(
  name= 'Reviewed Code',
  description= '''
    The reviewed code containing a list of review items and applied corrections. It is yet to be tested.
  ''',
)

tested_code = WorkflowArtifact(
  name= 'Tested Code',
  description= '''
    The tested code containing the finished code and proper tests. It is ready to be deployed.
  ''',
)

spec = WorkflowArtifact(
  name= 'Specification',
  description= '''
    The technical specification of the implemented code. It declares technical aspects
    for future programmers and can be translated to a manual.
  ''',
)

manual = WorkflowArtifact(
  name= 'Manual',
  description= '''
    A manual written for the customer so that they know how to properly use the changes made
    and can forward integrations to other departments or companies.
  ''',
)

### 6.3. Conversations

We have a total of 8 different conversations, 2 per phase:

- **Design (Modalities)**
- **Design (Language)**
- **Coding (Code)**
- **Coding (Design)**
- **Testing (Review)**
- **Testing (Test)**
- **Documenting (Specification)**
- **Documenting (Manual)**

In [27]:
design_modalities = WorkflowConversation(
  name= 'Design (Modalities)',
  description= '''
    The task given by the customer is analyzed and the respective modalities are created.
  ''',
  lead= ceo,
  assistant= cpo,
  input= task,
  output= modalities,
)

design_language = WorkflowConversation(
  name= 'Design (Language)',
  description= '''
    From the task and modalities specific use-cases are created that help the technical
    team to form working code from them.
  ''',
  lead= ceo,
  assistant= cto,
  input= modalities,
  output= language,
)

coding_code = WorkflowConversation(
  name= 'Coding (Code)',
  description= '''
    The initial implementation of the given use-cases realized as code.
    The code is not perfect yet, but it works.
    The code is always written in Python.
    Actual code is written and output in Markdown code blocks with the file name in bold above the block.
  ''',
  lead= cto,
  assistant= programmer,
  input= language,
  output= code,
)

coding_design = WorkflowConversation(
  name= 'Coding (Design)',
  description= '''
    The given code is analyzed for common architecturial problems and
    improved upon. Best practices are discussed and implemented by the programmer.
    After specifying problems the designer doesn't end the conversation directly, but
    lets the programmer correct the code (switch) and after corrections the programmer
    lets the designer end the conversation.
  ''',
  lead= programmer,
  assistant= designer,
  input= code,
  output= designed_code,
)

testing_review = WorkflowConversation(
  name= 'Testing (Review)',
  description= '''
    The completed code is reviewed for common code problems, errors
    and best practice problems. The programmer resolves problems.
    After specifying problems the reviewer doesn't end the conversation directly, but
    lets the programmer correct the code (switch) and after corrections the programmer
    lets the reviewer end the conversation.
  ''',
  lead= programmer,
  assistant= reviewer,
  input= designed_code,
  output= reviewed_code,
)

testing_test = WorkflowConversation(
  name= 'Testing (Test)',
  description= '''
    The given code is tested if it contains all business cases. Tests are written
    by the programmer by the test concept given by the tester.
  ''',
  lead= programmer,
  assistant= tester,
  input= reviewed_code,
  output= tested_code,
)

documenting_spec = WorkflowConversation(
  name= 'Documenting (Specification)',
  description= '''
    A technical specification is created that covers all aspects of the code
    so that following programmers and technical people can fully understand
    what it is doing.
  ''',
  lead= cto,
  assistant= programmer,
  input= tested_code,
  output= spec,
)

documenting_manual = WorkflowConversation(
  name= 'Documenting (Manual)',
  description= '''
    A business documentation is created for the customer. It explains
    what was implemented on a business level, which use-cases are covered
    and how integration into other systems can be done.
  ''',
  lead= ceo,
  assistant= cpo,
  input= spec,
  output= manual,
)

### 6.4. Phases

There are a total of 4 phases in our development workflow:

- **Designing**
- **Coding**
- **Testing**
- **Documenting**

In [28]:
designing = WorkflowPhase(
  name= 'Designing',
  description= '''
    The task is provided by the customer and business modalities
    and use-cases are created for the technical team to implement them properly.
  ''',
  conversations= [design_modalities, design_language],
)

coding = WorkflowPhase(
  name= 'Coding',
  description= '''
    The coding phase. Here the actual code implementation is created.
    Actual code is output in Markdown code blocks with the file name in bold above the block.
  ''',
  conversations= [coding_code, coding_design],
)

testing = WorkflowPhase(
  name= 'Testing',
  description= '''
    The testing phase. The implementation is reviewed and tested properly
    so that it checks all quality marks.
  ''',
  conversations= [testing_review, testing_test],
)

documenting = WorkflowPhase(
  name= 'Documenting',
  description= '''
    The implementation is documented for technical and business people
    so that it can be understood for future extensions, business usage and business integrations.
  ''',
  conversations= [documenting_spec, documenting_manual],
)

## 7. Starting the workflow

We put together our workflow manager with the data and OpenAI client we defined.

Then we use the `execute` method of the workflow manager to kick off our workflow.

We tell it to create some implementation. The exact prompt is as follows:

> 
> I want to create a web-based member portal that informs about events and news and also contains a blog
> 

Below the code you can see and follow the output.

In [29]:
manager = WorkflowManager(
  oai_client= oai_client,
  model= oai_model,
  assistant_id= oai_assistant_id,
  description= '''
    You will roleplay roles of a software company with a typical waterflow-based development workflow.
  ''',
  phases = [designing, coding, testing, documenting],
)

manager.execute(
  """
  I want to create a web-based member portal
  that informs about events and news and also
  contains a blog
  """
)


=== Phase Designing ===



=== Conversation Design (Modalities) ===


CEO:

We've received a task from our customer who would like to create a web-based member portal that informs about events and news, and also contains a blog. CPO, could you please start analyzing this task and create the respective modalities for our technical team? We need to ensure that we outline the right features and user interactions for such a platform.

=====

CPO:

Certainly, CEO. In order to design the modalities for the web-based member portal, I will outline the core features and interactions based on the task given. Here are the modalities we should consider:

1. **User Authentication**: Since it's a member portal, we should have a secure login and sign-up process to authenticate members.
2. **Events Information**: A section that lists upcoming events with details such as date, time, and location, as well as a feature for members to RSVP.
3. **News Updates**: A dynamic section where the latest news rel

RuntimeError: No new messages found in thread thread_DXTJz3BDXaWecFl8EmXqc6t0